Feature selection strategies can be divided into three main areas based on the type of strategy and
techniques employed for the same. They are described briefly as follows.
- **Filter methods**: These techniques select features purely based on metrics like
correlation, mutual information and so on. These methods do not depend on results
obtained from any model and usually check the relationship of each feature with
the response variable to be predicted. Popular methods include threshold based
methods and statistical tests.
- **Wrapper methods**: These techniques try to capture interaction between multiple
features by using a recursive approach to build multiple models using feature
subsets and select the best subset of features giving us the best performing model.
Methods like backward selecting and forward elimination are popular wrapper
based methods.
- **Embedded methods**: These techniques try to combine the benefits of the other
two methods by leveraging Machine Learning models themselves to rank and score
feature variables based on their importance. Tree based methods like decision trees
and ensemble methods like random forests are popular examples of embedded
methods.

In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
pt = np.get_printoptions()['threshold']

### Threshold-Based Methods
#### Limiting features in bag of word based models
The scikit-learn framework provides parameters like min_df and max_
df which can be used to specify thresholds for ignoring terms which have document frequency above and
below user specified thresholds

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0.1,max_df=.85,max_features=2000)
cv

#### Variance based thresholding

In [ ]:
df = pd.read_csv('datasets/Pokemon.csv')
poke_gen = pd.get_dummies(df['Generation'])
poke_gen.head()

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(.15)
vt.fit(poke_gen)

To view the variances as well as which features were finally selected by this algorithm, we can use the
variances_ property and the get_support(...) function respectively. The following snippet depicts this
clearly in a formatted dataframe

In [ ]:
pd.DataFrame({'variance': vt.variances_, 'select_feature': vt.get_support()},
             index=poke_gen.columns).T

We can clearly see which features have been selected based on their True values and also their variance
being above 0.15. To get the final subset of selected features, you can use the following code

In [ ]:
poke_gen_subset = poke_gen.iloc[:,vt.get_support()].head()
poke_gen_subset

### Statistical Methods

In [ ]:
from sklearn.datasets import load_breast_cancer
bc_data = load_breast_cancer()
bc_features = pd.DataFrame(bc_data.data, columns=bc_data.feature_names)
bc_classes = pd.DataFrame(bc_data.target, columns=['IsMalignant'])
# build featureset and response class labels
bc_X = np.array(bc_features)
bc_y = np.array(bc_classes).T[0]
print('Feature set shape:', bc_X.shape)
print('Response class shape:', bc_y.shape)

In [ ]:
np.set_printoptions(threshold=30)
print('Feature set data [shape: '+str(bc_X.shape)+']')
print(np.round(bc_X, 2), '\n')
print('Feature names:')
print(np.array(bc_features.columns), '\n')
print('Predictor Class label data [shape: '+str(bc_y.shape)+']')
print(bc_y, '\n')
print('Predictor name:', np.array(bc_classes.columns))
np.set_printoptions(threshold=pt)

Let’s now use the chi-square test on this feature set and select the top 15 best features
out of the 30 features. The following snippet helps us achieve this

In [ ]:
from sklearn.feature_selection import chi2, SelectKBest
skb = SelectKBest(score_func=chi2, k=15)
skb.fit(bc_X,bc_y)

In [ ]:
feature_scores = [(item, score) for item, score in zip(bc_data.feature_names, skb.scores_)]
sorted(feature_scores, key=lambda x: -x[1])[:10]

We can now create a subset of the 15 selected features obtained from our original feature set of 30
features with the help of the chi-square test by using the following code

In [ ]:
select_features_kbest = skb.get_support()
feature_names_kbest = bc_data.feature_names[select_features_kbest]
feature_subset_df = bc_features[feature_names_kbest]
bc_SX = np.array(feature_subset_df)
print(bc_SX.shape)
print(feature_names_kbest)

In [ ]:
# To view the new feature set, you can use the following snippet.
feature_subset_df.iloc[20:25]

Let’s now build a simple classification model using logistic regression on the original feature set of 30 features 
and compare the model accuracy performance with another model built using our selected 15 features. For model evaluation,
we will use the accuracy metric (percent of correct predictions) and use a five-fold cross-validation scheme.
The main idea here is to compare the model prediction performance between models trained on different feature sets

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
lr = LogisticRegression()
# evaluating accuracy for model built on full featureset
full_feat_acc = np.average(cross_val_score(lr, bc_X, bc_y, scoring='accuracy', cv=5))
# evaluating accuracy for model built on selected featureset
sel_feat_acc = np.average(cross_val_score(lr, bc_SX, bc_y, scoring='accuracy', cv=5))
print('Model accuracy statistics with 5-fold cross validation')
print('Model accuracy with complete feature set', bc_X.shape, ':', full_feat_acc)
print('Model accuracy with selected feature set', bc_SX.shape, ':', sel_feat_acc)

### Recursive Feature Elimination
Recursive Feature Elimination, also known as RFE, is a popular wrapper based feature selection technique,
which allows you to use this strategy. The basic idea is to start off with a specific Machine Learning estimator
like the Logistic Regression algorithm we used for our classification needs. Next we take the entire feature set
of 30 features and the corresponding response class variables. RFE aims to assign weights to these features
based on the model fit. Features with the smallest weights are pruned out and then a model is fit again on
the remaining features to obtain the new weights or scores. This process is recursively carried out multiple
times and each time features with the lowest scores/weights are eliminated, until the pruned feature subset
contains the desired number of features that the user wanted to select (this is taken as an input parameter at
the start). This strategy is also popularly known as backward elimination. Let’s select the top 15 features on
our breast cancer dataset now using RFE.

In [ ]:
from sklearn.feature_selection import RFE
lr = LogisticRegression()
rfe = RFE(estimator=lr, n_features_to_select=15, step=1)
rfe.fit(bc_X, bc_y)

In [ ]:
# We can now use the get_support(...) function to obtain the final 15 selected features
select_features_rfe = rfe.get_support()
feature_names_rfe = bc_data.feature_names[select_features_rfe]
print(feature_names_rfe)

In [ ]:
set(feature_names_kbest) & set(feature_names_rfe)

### Model based selection
Tree based models like decision trees and ensemble models like random forests (ensemble of trees) can be utilized not 
just for modeling alone but for feature selection. These models can be used to compute feature importances when building
the model that can in turn be used for selecting the best features and discarding irrelevant features with lower scores.
Random forest is an ensemble model. This can be used as an embedded feature selection method, where each decision tree 
model in the ensemble is built by taking a training sample of data from the entire dataset. This sample is a bootstrap 
sample (sample taken with replacement). Splits at any node are taken by choosing the best split from a random subset of
the features rather than taking all the features into account. This randomness tends to reduce the variance of the model
at the cost of slightly increasing the bias. Overall this produces a better and more generalized model. 
Let’s now use the random forest model to score and rank features based on their importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(bc_X, bc_y)

In [ ]:
# The following code uses this random forest estimator to score the features based on their importance
# and we display the top 10 most important features based on this score.
importance_scores = rfc.feature_importances_
feature_importances = [(feature, score) for feature, score in zip(bc_data.feature_names, importance_scores)]
sorted(feature_importances, key=lambda x: -x[1])[:10]

### Feature extraction using dimensionality reduction
#### Feature Extraction with Principal Component Analysis
Principal component analysis, popularly known as PCA, is a statistical method that uses the process of
linear, orthogonal transformation to transform a higher-dimensional set of features that could be possibly
correlated into a lower-dimensional set of linearly uncorrelated features. These transformed and newly
created features are also known as Principal Components or PCs. In any PCA transformation, the total
number of PCs is always less than or equal to the initial number of features. The first principal component
tries to capture the maximum variance of the original set of features. Each of the succeeding components
tries to capture more of the variance such that they are orthogonal to the preceding components. An
important point to remember is that PCA is sensitive to feature scaling.
Our main task is to take a set of initial features with dimension let’s say D and reduce it to a subset of
extracted principal components of a lower dimension LD. The matrix decomposition process of Singular
Value Decomposition is extremely useful in helping us obtain the principal components. 
Considering we have a data matrix $F_{n\times d} $, where we have n observations and
D dimensions (features), we can depict SVD of the feature matrix as $F_{n \times D} = USV^T$ such that all the principal
components are contained in the component V<sup>T</sup>, which can be depicted as follows:

${V^T}_{(D\times D)}=$
$\left[
    \begin{matrix}
        PC_{1(1\times D)} \\ 
        PC_{2(1\times D)} \\ 
        \vdots \\ 
        PC_{D(1\times D)} \\ 
    \end{matrix}
\right]
$

The principal components are represented by ${PC_1, PC_2, \cdots, PC_D}$ , which are all one-dimensional vectors
of dimensions (1 x D). For extracting the first d principal components, we can first transpose this matrix to
obtain the following representation.

$
PC_{(D\times D)}=(V^T)^T=[PC_{1(D\times 1)}|PC_{2(D\times 1)}|\cdots|PC_{D(D\times 1)}]
$

Let’s try to extract the first three principal components now from our breast cancer feature set of
30 features using SVD. We first center our feature matrix and then use SVD and subsetting to extract the first
three PCs using the following code

In [59]:
# center the feature set
bc_XC = bc_X - bc_X.mean(axis=0)
# decompose using SVD
U, S, VT = np.linalg.svd(bc_XC)
# get principal components
PC = VT.T
# get first 3 principal components
PC3 = PC[:, 0:3]
PC3.shape

(30, 3)

In [60]:
# reduce feature set dimensionality 
np.round(bc_XC.dot(PC3), 2)
# Thus you can see how powerful SVD and PCA can be in helping us reduce dimensionality by extracting
# necessary features. Of course in Machine Learning systems and pipelines you can use utilities from scikitlearn 
# instead of writing unnecessary code and equations. The following code enables us to perform PCA on
# our breast cancer feature set leveraging scikit-learn's APIs

array([[-1160.14,  -293.92,   -48.58],
       [-1269.12,    15.63,    35.39],
       [ -995.79,    39.16,     1.71],
       ...,
       [ -314.5 ,    47.55,    10.44],
       [-1124.86,    34.13,    19.74],
       [  771.53,   -88.64,   -23.89]])

In [61]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(bc_X)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [63]:
# To understand how much of the variance is explained by each of these principal components, you can use the following code.
pca.explained_variance_ratio_
# From the preceding output, as expected, we can see the maximum variance is explained by the first
# principal component. To obtain the reduced feature set, we can use the following snippet.

array([0.98204467, 0.01617649, 0.00155751])

In [64]:
bc_pca = pca.transform(bc_X)
np.round(bc_pca, 2)
# Let’s now quickly build a logistic regression model as before and use model accuracy and five-fold cross
# validation to evaluate the model quality using these three features.

array([[1160.14, -293.92,   48.58],
       [1269.12,   15.63,  -35.39],
       [ 995.79,   39.16,   -1.71],
       ...,
       [ 314.5 ,   47.55,  -10.44],
       [1124.86,   34.13,  -19.74],
       [-771.53,  -88.64,   23.89]])

In [65]:
np.average(cross_val_score(lr, bc_pca, bc_y, scoring='accuracy', cv=5))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.9280800307810695